# Steam Games Analysis

In [23]:
from dotenv import load_dotenv
import os
import pymysql
import pandas as pd

load_dotenv()

conn = pymysql.connect(host='localhost',
                        port=int(3306),
                        user='root',
                        passwd=os.getenv('MYSQL_PASSWORD'),
                        db='steam')

games_rating = pd.read_sql('SELECT * FROM vw_games_rating', conn)
user_top_games = pd.read_sql('''
                                SELECT
                                    vw_usergamesrating.steamid AS steamid,
                                    vw_usergamesrating.appid AS appid,
                                    title,
                                    vw_unique_game_genres.genre AS genre,
                                    price,
                                    avg_rating
                                FROM
                                    vw_usergamesrating
                                LEFT JOIN vw_games_avg_ratings ON
                                    vw_usergamesrating.appid = vw_games_avg_ratings.appid
                                LEFT JOIN app_id_info ON
                                    vw_games_avg_ratings.appid = app_id_info.appid
                                LEFT JOIN vw_unique_game_genres ON
                                    vw_games_avg_ratings.appid = vw_unique_game_genres.appid
                                WHERE
                                    vw_usergamesrating.steamid_appid_rnk = 1
                                    AND title IS NOT NULL
                                ''', conn)

In [33]:
games_rating.sample(5)


,appid,title,genre,price,avg_rating
636,55150,"Warhammer 40,000: Space Marine",Action,29.99,0.012465
2693,228940,Carnage Racing ™,Action,0.00,0.005833
2767,234920,Dyscourse,Indie,14.99,0.000100
2632,227320,You Need a Budget 4,Accounting,59.99,0.029801
1450,32900,Restaurant Empire II,Simulation,19.99,0.005008


In [34]:
user_top_games.sample(5)

,steamid,appid,title,genre,price,avg_rating
7696,76561197984881697,240,Counter-Strike: Source,Action,19.99,0.437956
10764,76561197974211151,220,Half-Life 2,Action,9.99,0.212280
21017,76561197972803452,41500,Torchlight,RPG,14.99,0.034230
139,76561197973759917,620,Portal 2,Action,19.99,0.044985
22978,76561197971051936,570,Dota 2,Action,0.00,0.164853
